In [1]:
# import packages and libraries

#import osmnx as ox
import geopandas as gpd
from pathlib import Path
import folium 


# init
@authors: [Alexandre Pereira Santos](alexandre.santos(at)lmu.de) & [Charlotta Mirbach](c.mirbach@lmu.de)

## data
- maximum flooded area (binarized)
- municipalities Rio grande do Sul (Brazil)

## tasks
- generate polygons for affected municipalities
- download OSM building for each affected municipality
- identify buildings within flooded area
- export to individual municipality buildings datasets


## functions

## preprocessing

### load input data

In [2]:
#input flooded area vector 
flood_path = Path('../data/external/')
flood_file = 'HYD_maximum_flood_extent_IPH_UFRGS_20240506_A.shp' 
flood_gdf = gpd.read_file(flood_path/flood_file)
#flood_gdf.drop(['gridcode','area'], axis=1, inplace=True)

# import the municipality boundaries
mun_path = Path('../data/external/')
mun_file = 'LIM_RS_municipalites_2022_A.shp'
mun_gdf = gpd.read_file(mun_path/mun_file)
mun_gdf = mun_gdf[(mun_gdf['NM_MUN']!='Lagoa dos Patos')]

In [3]:

#import building footprints
foot_path = Path('../../../Dropbox/x/PostDoc/02 colab and other/24 05 RS Floods/UFRGS/DADOS/')
foot_file = 'RS_URB_edificios_v05_IPH_A.shp'
foot_gdf = gpd.read_file(foot_path/foot_file) #, layer = 'rhguaiba_google_buildings'
foot_gdf.to_crs(flood_gdf.crs, inplace=True)

print('Flood CRS:', flood_gdf.crs,', Municipality CRS:', mun_gdf.crs, 'Buildings CRS:', foot_gdf.crs)

Flood CRS: EPSG:32722 , Municipality CRS: EPSG:32722 Buildings CRS: EPSG:32722


In [4]:
foot_gdf.to_crs(flood_gdf.crs, inplace=True)

### joining gdfs

In [5]:
join_gdf = foot_gdf.sjoin(mun_gdf, how='inner', predicate="within")
join_gdf.drop(['confidence','index_right','SIGLA_UF','AREA_KM2'], axis=1, inplace=True)
#affected_mun = join_gdf[join_gdf['index_right'].notna()].dissolve('NM_MUN') #.index_right.dropna(axis=0, subset=['index_right'], inplace=True)
#affected_mun.reset_index(inplace=True)
#affected_mun

### check join consistency

In [6]:
foot_gdf.head()

,fid,latitude,longitude,area_in_me,confidence,full_plus_,geometry
0,1.0,-28.848039,-51.572102,27.0563,0.6875,583C5C2H+Q5JG,"POLYGON ((444198.500 6808713.500, 444192.500 6..."
1,2.0,-30.026502,-51.051474,84.7454,0.7986,48XCXWFX+9CRX,"POLYGON ((495044.173 6678274.812, 495030.252 6..."
2,3.0,-28.805628,-51.295699,216.0186,0.8980,583C5PV3+PPVW,"POLYGON ((471152.134 6813507.314, 471146.282 6..."
3,4.0,-29.346574,-51.385515,7.1515,0.7107,582CMJ37+9QH4,"POLYGON ((462577.930 6753555.721, 462577.527 6..."
4,5.0,-30.667396,-51.931613,109.6640,0.8104,48XC83M9+29R3,"POLYGON ((410755.858 6606894.732, 410760.330 6..."


In [7]:
join_gdf.head()

,fid,latitude,longitude,area_in_me,full_plus_,geometry,CD_MUN,NM_MUN
0,1.0,-28.848039,-51.572102,27.0563,583C5C2H+Q5JG,"POLYGON ((444198.500 6808713.500, 444192.500 6...",4323309,Vila Flores
2870,2871.0,-28.880362,-51.496078,16.7416,583C4G93+VH3G,"POLYGON ((451626.273 6805171.279, 451628.676 6...",4323309,Vila Flores
4275,4276.0,-28.859079,-51.528961,18.3865,583C4FRC+9C7R,"POLYGON ((448410.267 6807509.061, 448407.051 6...",4323309,Vila Flores
6394,6395.0,-28.839627,-51.545011,20.6623,583C5F63+4XXW,"POLYGON ((446835.882 6809657.594, 446831.972 6...",4323309,Vila Flores
7671,7672.0,-28.860752,-51.548561,228.9957,583C4FQ2+MHWR,"POLYGON ((446503.439 6807306.307, 446493.856 6...",4323309,Vila Flores


In [8]:
print(join_gdf.shape, foot_gdf.shape)

(5847115, 8) (5857714, 7)


In [10]:
foot_len = len(foot_gdf.full_plus_.unique())

In [11]:
foot_len - foot_gdf.shape[0] # full_plus_code is not unique, but repeats in exceptional cases

-56

In [13]:
join_gdf.full_plus_.isna().describe() # there are no NA values in the full_plus_code in the joined df, though

count     5847115
unique          1
top         False
freq      5847115
Name: full_plus_, dtype: object

In [15]:
join_full_len = len(join_gdf.full_plus_.unique())

In [16]:
join_full_len - join_gdf.shape[0] # full_plus_code repeats in the same way it did for the foot_gdf, minus 1

-55

In [17]:
print('there are', foot_gdf.shape[0] - join_gdf.shape[0], 'less observations in the joined df than in the footprints df', 
      'this means', (foot_gdf.shape[0] - join_gdf.shape[0])/foot_gdf.shape[0] * 100, '%')
# less than half a percent seems fair enough

there are 10599 less observations in the joined df than in the footprints df this means 0.18094089264173704 %


### identifying buildings within the flooded area

In [19]:
flood_gdf.columns

Index(['fid', 'DN', 'geometry'], dtype='object')

In [20]:
# eliminating unnecessary columns in the flooded area gdf
# these columns are all filled with a single value, probably a residue from the previous analyis process
#flood_gdf.drop(['CD_MUN', 'NM_MUN', 'SIGLA_UF', 'bacia', 'MUN_KM2', 'area_km2'], axis=1, inplace=True)
flood_gdf['flooded'] = 1
flood_gdf


,fid,DN,geometry,flooded
0,1.0,1,"POLYGON ((474897.943 6858472.962, 474924.342 6...",1
1,2.0,1,"POLYGON ((287987.640 6842194.198, 288093.148 6...",1
2,3.0,1,"POLYGON ((287828.837 6842221.204, 287881.592 6...",1
3,4.0,1,"POLYGON ((288253.566 6842079.492, 288279.942 6...",1
4,5.0,1,"POLYGON ((288386.527 6842022.137, 288439.280 6...",1
...,...,...,...,...
18369,27112.0,1,"POLYGON ((291003.460 6631527.324, 291029.348 6...",1
18370,27113.0,1,"POLYGON ((288523.977 6631180.458, 288575.753 6...",1
18371,27114.0,1,"POLYGON ((290612.829 6631639.342, 290845.827 6...",1
18372,27117.0,1,"POLYGON ((291426.856 6631057.294, 291478.631 6...",1


In [23]:
flooded_join_gdf  = join_gdf.sjoin(flood_gdf, how='left', predicate="intersects")
flooded_join_gdf.flooded.fillna(0,inplace=True)

#flooded_join_gdf.columns = ['latitude', 'longitude', 'area_m2', 'full_code', 'geometry','CD_MUN', 'NM_MUN', 'flooded']

In [24]:
flooded_join_gdf.columns

Index(['fid_left', 'latitude', 'longitude', 'area_in_me', 'full_plus_',
       'geometry', 'CD_MUN', 'NM_MUN', 'index_right', 'fid_right', 'DN',
       'flooded'],
      dtype='object')

In [25]:
flooded_join_gdf.drop(['index_right','fid_right','DN'], axis=1, inplace=True)

In [26]:
# checking for losses in the join

flooded_join_gdf.shape[0] - join_gdf.shape[0]

179

### export individual shapefiles for the affected municipalities

In [27]:
# identifying the municipalities affected by the flood

join_mun_gdf = mun_gdf.sjoin(flood_gdf,how='left',)
affected_mun = join_mun_gdf[join_mun_gdf['index_right'].notna()].dissolve('NM_MUN') #.index_right.dropna(axis=0, subset=['index_right'], inplace=True)
affected_mun.reset_index(inplace=True)
affected_mun.drop(['index_right','SIGLA_UF'], axis=1, inplace=True)
affected_mun

,NM_MUN,geometry,CD_MUN,AREA_KM2,fid,DN,flooded
0,Agudo,"POLYGON ((279918.986 6703043.270, 279741.282 6...",4300109,534.624,20308.0,1.0,1.0
1,Alto Alegre,"POLYGON ((304509.258 6807218.192, 304473.994 6...",4300554,115.335,2794.0,1.0,1.0
2,Alvorada,"POLYGON ((492898.701 6675799.152, 492877.273 6...",4300604,71.700,25966.0,1.0,1.0
3,Anta Gorda,"POLYGON ((399132.817 6788513.969, 399079.336 6...",4300703,242.100,5188.0,1.0,1.0
4,Araricá,"POLYGON ((508478.068 6723485.678, 508462.897 6...",4300877,35.391,19209.0,1.0,1.0
...,...,...,...,...,...,...,...
152,Vespasiano Corrêa,"POLYGON ((422599.491 6779596.277, 422602.356 6...",4322855,113.622,8010.0,1.0,1.0
153,Viamão,"POLYGON ((493121.166 6661507.218, 492956.456 6...",4323002,1496.506,27055.0,1.0,1.0
154,Victor Graeff,"POLYGON ((337313.030 6851935.042, 337403.314 6...",4323200,238.133,1101.0,1.0,1.0
155,Vila Maria,"POLYGON ((388212.410 6833956.117, 387963.159 6...",4323408,181.061,248.0,1.0,1.0


### creating individual shapefiles for each municipality

In [28]:
#setting the environment

AOI_path = Path('../data/processed/')
mun_file_list = []

# clipping the buildings to each affected municipality and saving it to a shapefile

for mun in affected_mun.iloc[0:10].iterrows():
    #print(mun[1].geometry)
    roi_gdf = gpd.GeoDataFrame(geometry=[mun[1].geometry],crs=mun_gdf.crs)
    mun_buildings = flooded_join_gdf.clip(roi_gdf)
    foot_file = 'RS_URB_' + mun[1].NM_MUN + '_affected_buildings_A.shp'
    mun_file_list.append(foot_file)
    mun_buildings.to_file(AOI_path / foot_file)

In [29]:
for mun in affected_mun.iloc[10:20].iterrows():
    #print(mun[1].geometry)
    roi_gdf = gpd.GeoDataFrame(geometry=[mun[1].geometry],crs=mun_gdf.crs)
    mun_buildings = flooded_join_gdf.clip(roi_gdf)
    foot_file = 'RS_URB_' + mun[1].NM_MUN + '_affected_buildings_A.shp'
    mun_file_list.append(foot_file)
    mun_buildings.to_file(AOI_path / foot_file)

In [30]:
for mun in affected_mun.iloc[20:30].iterrows():
    #print(mun[1].geometry)
    roi_gdf = gpd.GeoDataFrame(geometry=[mun[1].geometry],crs=mun_gdf.crs)
    mun_buildings = flooded_join_gdf.clip(roi_gdf)
    foot_file = 'RS_URB_' + mun[1].NM_MUN + '_affected_buildings_A.shp'
    mun_file_list.append(foot_file)
    mun_buildings.to_file(AOI_path / foot_file)

In [31]:
for mun in affected_mun.iloc[30:40].iterrows():
    #print(mun[1].geometry)
    roi_gdf = gpd.GeoDataFrame(geometry=[mun[1].geometry],crs=mun_gdf.crs)
    mun_buildings = flooded_join_gdf.clip(roi_gdf)
    foot_file = 'RS_URB_' + mun[1].NM_MUN + '_affected_buildings_A.shp'
    mun_file_list.append(foot_file)
    mun_buildings.to_file(AOI_path / foot_file)

In [32]:
for mun in affected_mun.iloc[40:50].iterrows():
    #print(mun[1].geometry)
    roi_gdf = gpd.GeoDataFrame(geometry=[mun[1].geometry],crs=mun_gdf.crs)
    mun_buildings = flooded_join_gdf.clip(roi_gdf)
    foot_file = 'RS_URB_' + mun[1].NM_MUN + '_affected_buildings_A.shp'
    mun_file_list.append(foot_file)
    mun_buildings.to_file(AOI_path / foot_file)

In [33]:
for mun in affected_mun.iloc[50:60].iterrows():
    #print(mun[1].geometry)
    roi_gdf = gpd.GeoDataFrame(geometry=[mun[1].geometry],crs=mun_gdf.crs)
    mun_buildings = flooded_join_gdf.clip(roi_gdf)
    foot_file = 'RS_URB_' + mun[1].NM_MUN + '_affected_buildings_A.shp'
    mun_file_list.append(foot_file)
    mun_buildings.to_file(AOI_path / foot_file)

In [34]:
for mun in affected_mun.iloc[60:70].iterrows():
    #print(mun[1].geometry)
    roi_gdf = gpd.GeoDataFrame(geometry=[mun[1].geometry],crs=mun_gdf.crs)
    mun_buildings = flooded_join_gdf.clip(roi_gdf)
    foot_file = 'RS_URB_' + mun[1].NM_MUN + '_affected_buildings_A.shp'
    mun_file_list.append(foot_file)
    mun_buildings.to_file(AOI_path / foot_file)

C:\Users\ra42zek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\geopandas\io\file.py:612: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


In [35]:
for mun in affected_mun.iloc[70:78].iterrows():
    #print(mun[1].geometry)
    roi_gdf = gpd.GeoDataFrame(geometry=[mun[1].geometry],crs=mun_gdf.crs)
    mun_buildings = flooded_join_gdf.clip(roi_gdf)
    foot_file = 'RS_URB_' + mun[1].NM_MUN + '_affected_buildings_A.shp'
    mun_file_list.append(foot_file)
    mun_buildings.to_file(AOI_path / foot_file)

In [36]:
# building a single export run as geopackage
# WARNING: this can take a while (not too bad, ca.12 min)

for mun in affected_mun.iloc[0:156].iterrows():
    roi_gdf = gpd.GeoDataFrame(geometry=[mun[1].geometry],crs=mun_gdf.crs)
    mun_buildings = flooded_join_gdf.clip(roi_gdf)
    foot_file = 'RS_URB_' + mun[1].NM_MUN + '_affected_buildings_A.gpkg'
    mun_file_list.append(foot_file)
    mun_buildings.to_file(AOI_path / foot_file, driver="GPKG")

C:\Users\ra42zek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\geopandas\io\file.py:612: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


In [37]:
affected_mun.shape

(157, 7)

In [38]:
for mun in affected_mun.iloc[78:157].iterrows():
    roi_gdf = gpd.GeoDataFrame(geometry=[mun[1].geometry],crs=mun_gdf.crs)
    mun_buildings = flooded_join_gdf.clip(roi_gdf)
    foot_file = 'RS_URB_' + mun[1].NM_MUN + '_affected_buildings_A.gpkg'
    mun_file_list.append(foot_file)
    mun_buildings.to_file(AOI_path / foot_file, driver="GPKG")